In [2]:
import model
from utils.data_utils import Season
from utils.model_utils import Feature
from data.load_footystats import GenerateDataFrame
from data.load_csv import DataLoader

ModuleNotFoundError: No module named 'utils'

In [ ]:
table = train.X.describe().transpose()[['mean', 'min', 'max', 'std']]
# save table to csv file
table.to_csv('table.csv')
exit()